In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import seaborn as sns
import re
import random as rd
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from nltk import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords, wordnet
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import autokeras as ak
from sklearn.model_selection import KFold

import nltk
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ddayv\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ddayv\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
def Tokenize(f):     ## Pre-processando a frase

    ## Colocando em minusculo
    ## Retirando a pontuaçao
    ## Retirando as StopWords
    
    f = f.lower().replace('\n', '').replace('-','').replace('#','').replace('.','').replace(',','').replace('!','').replace('r\n','').replace('  ','').replace('https','').replace('rt','').replace('rn','')
    token = RegexpTokenizer(r"\w+")
    f = token.tokenize(f)
    
    stop_words = set(stopwords.words('portuguese'))
    
    new_word = [word for word in f if not word in stop_words]
    
    return ' '.join(new_word)

def remove_user(frase):
    frase = re.sub('@\w+','',frase)
    frase = re.sub('{https}[^ ]+','',frase)
    frase = re.sub('https\w+','',frase)
    # re.sub('#\w+','',frase)
    return frase

def pre_X(frases):
    lista = []
    
    for frase in frases:
        lista.append(frase)
        
    return lista

def pre_Y(number):
    lista = []
    
    for numb in number:
        lista.append(numb)
    
    return lista

def set_array(frases):
    
    vocab = []
    palavras = []
    for frase in frases:
        
        text_array = remove_user(frase)
        text_array = Tokenize(text_array)
        text_array = text_array.split(' ')
        for i in range(len(text_array)):
            vocab.append(text_array[i])
        
    return vocab


## TWEETSENT

In [121]:
df_tweet= pd.read_csv('export_TweetSentBR.csv')
df_tweet

,id,id_twitter,text,sentiment
0,1343,863044774588272640,Que coisa linda! O Programa #encontro estava m...,1
1,1344,865583716088766467,"Por mais #Encontro com as Irmãs Galvão, adorei...",1
2,1345,865063232201011201,Mr. CATRA @OficialMrCatra lançando sua nova mú...,1
3,1346,864668391008763905,quem viu aquela lutadora modela barbuda tatuad...,0
4,1347,865572794016378882,Tô passada com esse cara.... quanta merda pode...,-1
...,...,...,...,...
11565,12908,864636619000877056,eu ja to aqui pronto pro #MasterChefBR mas ain...,-1
11566,12909,863581588713603072,MALUCO! Uma coisa que eu não tenho coragem é e...,-1
11567,12910,864831041349054464,#MaisVoce @ANAMARIABRAGA está linda @RedeGlobo,1
11568,12911,863042798575951872,"Que orgulho de ti, @sportrecife! #Encontro",1


In [122]:
df_remove = df_tweet[df_tweet['sentiment'] == '-']
df_tweet = df_tweet.drop(df_remove.index)

df_tweet['text'] = df_tweet['text'].apply(remove_user)
df_tweet['text'] = df_tweet['text'].apply(Tokenize)

In [123]:
df_tweet['sentiment'] = df_tweet['sentiment'].apply(lambda x: int(x))

In [124]:
text_Tweetsent = df_tweet['text']
polarity_ = np.asarray(df_tweet['sentiment'])

In [125]:
count_vect = CountVectorizer()
X_text = count_vect.fit_transform(text_Tweetsent)
tfidf_transformer = TfidfTransformer()
X_train_TWEET = tfidf_transformer.fit_transform(X_text) # Aplicando o TF-IDF
X_train_TWEET.shape

(11533, 15334)

## TASH

In [126]:
df1 = pd.read_csv("tash-pt.csv")
df1 = df1.dropna()
df1 = df1.drop(columns=['id_twitter'])


In [127]:
df1['text'] = df1['text'].apply(remove_user)
df1['text'] = df1['text'].apply(Tokenize)

In [128]:
text_TASH = df1['text']
sentiment_ = np.asarray(df1['sentiment'])

X_text_TASH = count_vect.transform(text_TASH)
X_text_TASH_ = tfidf_transformer.transform(X_text_TASH) # Aplicando o TF-IDF
X_text_TASH_.shape

(2787, 15334)

## KANSOAN

In [129]:
df = pd.read_csv("data.txt", sep=',', header=None, names=['text','sentiment'])

df_remove = df[df['sentiment'] == '#Inveja'].index
df = df.drop(df_remove)

df_remove = df[df['sentiment'] == '#Raiva'].index
df = df.drop(df_remove)

df_remove = df[df['sentiment'] == '#Ironia'].index
df = df.drop(df_remove)


def binario(termo):
    if termo == '#Feliz' or termo == '#Amor':
        return 1
    elif termo == '#Triste' or termo == '#Chateado':
        return 0       
    
df['sentiment'] = df['sentiment'].apply(binario)

In [130]:
df['text'] = df['text'].apply(remove_user)
df['text'] = df['text'].apply(Tokenize)

In [131]:
text_KANSOAN = df['text']
sentiment = np.asarray(df['sentiment'])

X_text_KANSOAN = count_vect.transform(text_KANSOAN)
X_text_KANSOAN_ = tfidf_transformer.transform(X_text_KANSOAN) # Aplicando o TF-IDF
X_text_KANSOAN_.shape

(9224, 15334)

## UNILEX

In [132]:
df1 = pd.read_excel('./rotulação/TweetsPolitical01 OK OK.xlsx')
df2 = pd.read_excel('./rotulação/TweetsPolitical02 OK OK.xlsx')
df3 = pd.read_excel('./rotulação/TweetsPolitical03 OK OK.xlsx')
df4 = pd.read_excel('./rotulação/TweetsPolitical04 OK OK.xlsx')
df5 = pd.read_excel('./rotulação/TweetsPolitical05 OK OK.xlsx')
df6 = pd.read_excel('./rotulação/TweetsPolitical06 OK OK.xlsx')
df7 = pd.read_excel('./rotulação/TweetsPolitical07 OK OK.xlsx')
df8 = pd.read_excel('./rotulação/TweetsPolitical08 OK OK.xlsx')
df9 = pd.read_excel('./rotulação/TweetsPolitical09 OK OK.xlsx')
df10 = pd.read_excel('./rotulação/TweetsPolitical10 OK OK.xlsx')
df11 = pd.read_excel('./rotulação/TweetsPolitical11 OK OK.xlsx')
df12 = pd.read_excel('./rotulação/TweetsPolitical12 OK OK.xlsx')
df13 = pd.read_excel('./rotulação/TweetsPolitical13 OK OK.xlsx')
df14 = pd.read_excel('./rotulação/TweetsPolitical14 OK OK.xlsx')
df15 = pd.read_excel('./rotulação/TweetsPolitical15 OK OK.xlsx')

lista = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15]

df_all = pd.concat(lista)
df_all.shape

(12670, 2)

In [133]:
df_all = df_all.dropna()

In [134]:
df_all.drop(df_all[df_all['Polaridade'] == 11].index, inplace=True)
df_all.drop(df_all[df_all['Polaridade'] == -2].index, inplace=True)
df_all.drop(df_all[df_all['Polaridade'] == 10].index, inplace=True)

df_all['Polaridade'].unique()

array([ 0.,  1., -1.])

In [135]:
df_all.head()

,Tweet,Polaridade
0,- #CaoNossoDeCadaDia #Novo - Vanessa Mandotti ...,0.0
1,"- Bola pra frente que amanhã é outro dia, outr...",1.0
2,- Cara de mal? Acho que não... Apenas um corte...,1.0
3,""" #PCdoB O PARTIDO QUE FAZ A DIFERENÇA: LEAL E...",1.0
4,"""@camaradamae: #PCdoB o Partido da coragem ! ...",1.0


In [136]:
df_all['Tweet'] = df_all['Tweet'].apply(remove_user)
df_all['Tweet'] = df_all['Tweet'].apply(Tokenize)
polarity = np.asarray(df_all['Polaridade'])
Tweet = df_all['Tweet']


In [137]:
Tweet = count_vect.transform(Tweet)
X_text_UNILEX = tfidf_transformer.transform(Tweet) # Aplicando o TF-IDF
X_text_UNILEX.shape

(12626, 15334)

## NB - TRAIN -> TWEETSENT- TEST -> UNILEX

In [138]:
## TIRAR OS SPLITS , até no treino

In [139]:
clf = MultinomialNB()
clf.fit(X_train_TWEET, polarity_)
y_pred = clf.predict(X_text_UNILEX)

print(classification_report(polarity,y_pred))

              precision    recall  f1-score   support

        -1.0       0.50      0.44      0.47      4186
         0.0       0.37      0.22      0.28      4740
         1.0       0.35      0.58      0.44      3700

    accuracy                           0.40     12626
   macro avg       0.41      0.41      0.39     12626
weighted avg       0.41      0.40      0.39     12626



## NB - TRAIN -> TWEETSENT - TEST -> TASH

In [140]:
clf = MultinomialNB()
clf.fit(X_train_TWEET, polarity_)
y_pred = clf.predict(X_text_TASH_)

print(classification_report(sentiment_,y_pred))

              precision    recall  f1-score   support

          -1       0.45      0.48      0.46       881
           0       0.46      0.08      0.13      1018
           1       0.39      0.73      0.51       888

    accuracy                           0.41      2787
   macro avg       0.43      0.43      0.37      2787
weighted avg       0.43      0.41      0.35      2787



## NB - TRAIN -> TWEET - TEST -> KANSOAN

In [141]:
def func(num):
    if num == 0:
        return -1
    else:
        return 1

sentiment = df['sentiment'].apply(func)


In [144]:
clf = MultinomialNB()
clf.fit(X_train_TWEET, polarity_)
y_pred = clf.predict(X_text_KANSOAN_)

print(classification_report(sentiment,y_pred,zero_division=True))

              precision    recall  f1-score   support

          -1       0.94      0.76      0.84      4504
           0       0.00      1.00      0.00         0
           1       0.82      0.94      0.87      4720

    accuracy                           0.85      9224
   macro avg       0.59      0.90      0.57      9224
weighted avg       0.88      0.85      0.86      9224



## SVM - TRAIN->TWEETSENT - TEST-> UNILEX

In [145]:
clf = SVC().fit(X_train_TWEET, polarity_) 
y_pred = clf.predict(X_text_UNILEX)

print(classification_report(polarity,y_pred))

              precision    recall  f1-score   support

        -1.0       0.52      0.37      0.43      4186
         0.0       0.37      0.25      0.30      4740
         1.0       0.34      0.59      0.43      3700

    accuracy                           0.39     12626
   macro avg       0.41      0.40      0.39     12626
weighted avg       0.41      0.39      0.38     12626



## SVM T TWEET T TASH

In [146]:
clf = SVC().fit(X_train_TWEET, polarity_) 
y_pred = clf.predict(X_text_TASH_)

print(classification_report(sentiment_,y_pred))

              precision    recall  f1-score   support

          -1       0.43      0.50      0.46       881
           0       0.43      0.12      0.18      1018
           1       0.40      0.66      0.50       888

    accuracy                           0.41      2787
   macro avg       0.42      0.43      0.38      2787
weighted avg       0.42      0.41      0.37      2787



## SVM T TWEET T KANSOAN

In [147]:
clf = SVC().fit(X_train_TWEET, polarity_) 
y_pred = clf.predict(X_text_KANSOAN_)

print(classification_report(sentiment,y_pred, zero_division=True))

              precision    recall  f1-score   support

          -1       0.93      0.74      0.83      4504
           0       0.00      1.00      0.00         0
           1       0.82      0.91      0.86      4720

    accuracy                           0.83      9224
   macro avg       0.58      0.88      0.56      9224
weighted avg       0.87      0.83      0.85      9224



## RL T TWEETSENT T UNILEX

In [148]:
clf = LogisticRegression(max_iter=1000).fit(X_train_TWEET, polarity_) 
y_pred = clf.predict(X_text_UNILEX)

print(classification_report(polarity,y_pred))

              precision    recall  f1-score   support

        -1.0       0.49      0.44      0.47      4186
         0.0       0.40      0.37      0.38      4740
         1.0       0.36      0.44      0.40      3700

    accuracy                           0.41     12626
   macro avg       0.42      0.42      0.41     12626
weighted avg       0.42      0.41      0.41     12626



## RL T TWEET T TASH

In [149]:
clf = LogisticRegression(max_iter=1000).fit(X_train_TWEET, polarity_) 
y_pred = clf.predict(X_text_TASH_)

print(classification_report(sentiment_,y_pred))

              precision    recall  f1-score   support

          -1       0.42      0.57      0.48       881
           0       0.44      0.19      0.27      1018
           1       0.43      0.56      0.49       888

    accuracy                           0.43      2787
   macro avg       0.43      0.44      0.41      2787
weighted avg       0.43      0.43      0.41      2787



## RL T TWEET T KANSOAN

In [150]:
clf = LogisticRegression(max_iter=1000).fit(X_train_TWEET, polarity_) 
y_pred = clf.predict(X_text_KANSOAN_)

print(classification_report(sentiment,y_pred, zero_division=True))

              precision    recall  f1-score   support

          -1       0.92      0.80      0.86      4504
           0       0.00      1.00      0.00         0
           1       0.88      0.88      0.88      4720

    accuracy                           0.84      9224
   macro avg       0.60      0.90      0.58      9224
weighted avg       0.90      0.84      0.87      9224



In [155]:
one = OneHotEncoder(sparse=False)
teste_ = one.fit_transform(np.asarray(sentiment).reshape(-1,1))
nome = pd.DataFrame(teste_)
nome['coluna'] = 0
nome.rename(columns={1: 2, 'coluna':1} , inplace=True)
Y_KANSOAN = nome[[0,1,2]].to_numpy()

## TOTALMENT.C T TWEETSENT T UNILEX

In [151]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Dense(25, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Dense(10, activation='tanh'),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Dense(3 , activation='softmax')
])

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=['accuracy']
)

y_one = one.fit_transform(np.asarray(polarity_).reshape(-1,1))
y_one_test = one.fit_transform(polarity.reshape(-1,1))

fit = model.fit(X_train_TWEET.todense(), y_one, epochs=5, validation_data=(X_text_UNILEX.todense(), y_one_test))

Epoch 1/5
361/361 [==============================] - 4s 10ms/step - loss: 0.9880 - accuracy: 0.5098 - val_loss: 1.0657 - val_accuracy: 0.4084
Epoch 2/5
361/361 [==============================] - 2s 7ms/step - loss: 0.7186 - accuracy: 0.7029 - val_loss: 1.1343 - val_accuracy: 0.4498
Epoch 3/5
361/361 [==============================] - 2s 7ms/step - loss: 0.4664 - accuracy: 0.8392 - val_loss: 1.3982 - val_accuracy: 0.4473
Epoch 4/5
361/361 [==============================] - 2s 6ms/step - loss: 0.3002 - accuracy: 0.9034 - val_loss: 1.7784 - val_accuracy: 0.4384
Epoch 5/5
361/361 [==============================] - 2s 6ms/step - loss: 0.2073 - accuracy: 0.9377 - val_loss: 2.0712 - val_accuracy: 0.4316


In [152]:
predicted = model.predict(X_text_UNILEX.todense())

print(classification_report(np.argmax(y_one_test, axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.49      0.38      0.43      4186
           1       0.41      0.52      0.46      4740
           2       0.41      0.38      0.39      3700

    accuracy                           0.43     12626
   macro avg       0.44      0.43      0.43     12626
weighted avg       0.44      0.43      0.43     12626



## TASH

In [153]:
predicted = model.predict(X_text_TASH_.todense())

print(classification_report(np.argmax(one.fit_transform(sentiment_.reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.41      0.49      0.45       881
           1       0.41      0.27      0.32      1018
           2       0.42      0.50      0.46       888

    accuracy                           0.41      2787
   macro avg       0.41      0.42      0.41      2787
weighted avg       0.41      0.41      0.41      2787



## KANSOAN

In [156]:
predicted = model.predict(X_text_KANSOAN_.todense())

print(classification_report(np.argmax(Y_KANSOAN, axis=1), np.argmax(predicted, axis=1), zero_division=True))

              precision    recall  f1-score   support

           0       0.89      0.80      0.84      4504
           1       0.00      1.00      0.00         0
           2       0.91      0.78      0.84      4720

    accuracy                           0.79      9224
   macro avg       0.60      0.86      0.56      9224
weighted avg       0.90      0.79      0.84      9224



## LSTM T TWEET T TASH

In [157]:
vectorize_layer = TextVectorization(
 max_tokens=15000,
 output_mode='int',
 output_sequence_length=len(max(df_tweet['text'] )))

vocab = set_array(df_tweet['text'] )

vectorize_layer.adapt(np.unique(vocab))
len(vectorize_layer.get_vocabulary())


15000

In [160]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(
        input_dim=len(vectorize_layer.get_vocabulary()),
        output_dim=64,mask_zero=True),
    
    tf.keras.layers.LSTM(50, activation='relu' ,return_sequences=True),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.LSTM(25 , activation='tanh', return_sequences=True),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.LSTM(10 , activation='tanh'),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(3, activation='softmax')
])


model.compile(
    optimizer= tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=['accuracy']
)

y_one = one.fit_transform(np.asarray(polarity_).reshape(-1,1))
y_one_test = one.fit_transform(np.asarray(sentiment_).reshape(-1,1))

fit = model.fit(np.asarray(pre_X(text_Tweetsent)), y_one, epochs=5, batch_size=128 ,validation_data=(np.asarray(pre_X(text_TASH)), y_one_test))

Epoch 1/5
91/91 [==============================] - 16s 171ms/step - loss: 1.0456 - accuracy: 0.4725 - val_loss: 1.2170 - val_accuracy: 0.3330
Epoch 2/5
91/91 [==============================] - 14s 154ms/step - loss: 0.8343 - accuracy: 0.6165 - val_loss: 1.1322 - val_accuracy: 0.4288
Epoch 3/5
91/91 [==============================] - 14s 153ms/step - loss: 0.6328 - accuracy: 0.7468 - val_loss: 1.2779 - val_accuracy: 0.4370
Epoch 4/5
91/91 [==============================] - 14s 158ms/step - loss: 0.4522 - accuracy: 0.8399 - val_loss: 1.6150 - val_accuracy: 0.4306
Epoch 5/5
91/91 [==============================] - 14s 159ms/step - loss: 0.3418 - accuracy: 0.8874 - val_loss: 1.8509 - val_accuracy: 0.4291


In [161]:
predicted = model.predict(np.asarray(pre_X(text_TASH)))

print(classification_report(np.argmax(one.fit_transform(np.asarray(sentiment_).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.44      0.33      0.38       881
           1       0.41      0.66      0.50      1018
           2       0.49      0.26      0.34       888

    accuracy                           0.43      2787
   macro avg       0.45      0.42      0.41      2787
weighted avg       0.44      0.43      0.41      2787



## UNILEX

In [162]:
predicted = model.predict(np.asarray(pre_X(df_all['Tweet'])))

print(classification_report(np.argmax(one.fit_transform(np.asarray(polarity).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.55      0.20      0.29      4186
           1       0.41      0.87      0.55      4740
           2       0.55      0.14      0.22      3700

    accuracy                           0.43     12626
   macro avg       0.50      0.40      0.36     12626
weighted avg       0.49      0.43      0.37     12626



## KANSOAN

In [167]:
predicted = model.predict(np.asarray(pre_X(text_KANSOAN)))

print(classification_report(np.argmax(Y_KANSOAN, axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.86      0.55      0.67      4504
           1       0.00      0.00      0.00         0
           2       0.87      0.39      0.54      4720

    accuracy                           0.47      9224
   macro avg       0.57      0.32      0.40      9224
weighted avg       0.86      0.47      0.61      9224



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## CONV1D T TWEET T KANSOAN

In [168]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(
        input_dim=len(vectorize_layer.get_vocabulary()),
        output_dim=64,
        mask_zero=True),
    
    tf.keras.layers.Conv1D(32,6, activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')   
])

model.compile(
    optimizer= tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=['accuracy']
)

y_one = one.fit_transform(np.asarray(polarity_).reshape(-1,1))
y_one_test = Y_KANSOAN

fit = model.fit(np.asarray(pre_X(text_Tweetsent)), y_one, epochs=5, batch_size=128, validation_data=(np.asarray(pre_X(text_KANSOAN)),Y_KANSOAN))

Epoch 1/5
91/91 [==============================] - 3s 33ms/step - loss: 1.0366 - accuracy: 0.4750 - val_loss: 0.9644 - val_accuracy: 0.5652
Epoch 2/5
91/91 [==============================] - 3s 32ms/step - loss: 0.8521 - accuracy: 0.6116 - val_loss: 0.7054 - val_accuracy: 0.8071
Epoch 3/5
91/91 [==============================] - 3s 32ms/step - loss: 0.6628 - accuracy: 0.6880 - val_loss: 0.5843 - val_accuracy: 0.8110
Epoch 4/5
91/91 [==============================] - 3s 33ms/step - loss: 0.5267 - accuracy: 0.7643 - val_loss: 0.5692 - val_accuracy: 0.7535
Epoch 5/5
91/91 [==============================] - 3s 31ms/step - loss: 0.4133 - accuracy: 0.8486 - val_loss: 0.7584 - val_accuracy: 0.6333


In [169]:
predicted = model.predict(np.asarray(pre_X(text_KANSOAN)))

print(classification_report(np.argmax(Y_KANSOAN, axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.94      0.74      0.83      4504
           1       0.00      0.00      0.00         0
           2       0.95      0.53      0.68      4720

    accuracy                           0.63      9224
   macro avg       0.63      0.42      0.50      9224
weighted avg       0.95      0.63      0.75      9224



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## UNILEX

In [170]:
predicted = model.predict(np.asarray(pre_X(df_all['Tweet'])))

print(classification_report(np.argmax(one.fit_transform(np.asarray(polarity).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.53      0.43      0.48      4186
           1       0.43      0.71      0.54      4740
           2       0.49      0.20      0.29      3700

    accuracy                           0.47     12626
   macro avg       0.49      0.45      0.43     12626
weighted avg       0.48      0.47      0.44     12626



## TASH

In [171]:
predicted = model.predict(np.asarray(pre_X(text_TASH)))

print(classification_report(np.argmax(one.fit_transform(np.asarray(sentiment_).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.42      0.37      0.40       881
           1       0.37      0.53      0.44      1018
           2       0.46      0.29      0.35       888

    accuracy                           0.40      2787
   macro avg       0.42      0.40      0.40      2787
weighted avg       0.42      0.40      0.40      2787



## BDR T TWEET T UNILEX

In [174]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(
        input_dim=len(vectorize_layer.get_vocabulary()),
        output_dim=1,
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(1)),

    tf.keras.layers.Dense(3, activation='softmax')
])

model.compile(
    optimizer= tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=['accuracy']
)

y_one = one.fit_transform(np.asarray(polarity_).reshape(-1,1))
y_one_test = one.fit_transform(np.asarray(polarity).reshape(-1,1))

fit = model.fit(np.asarray(pre_X(text_Tweetsent)), y_one, epochs=20, batch_size=128 ,validation_data=(np.asarray(pre_X(df_all['Tweet'])), y_one_test))

Epoch 1/10
91/91 [==============================] - 5s 59ms/step - loss: 1.0823 - accuracy: 0.4491 - val_loss: 1.1138 - val_accuracy: 0.2930
Epoch 2/10
91/91 [==============================] - 3s 38ms/step - loss: 1.0556 - accuracy: 0.4530 - val_loss: 1.0931 - val_accuracy: 0.3587
Epoch 3/10
91/91 [==============================] - 4s 40ms/step - loss: 1.0119 - accuracy: 0.4740 - val_loss: 1.0861 - val_accuracy: 0.3817
Epoch 4/10
91/91 [==============================] - 5s 58ms/step - loss: 0.9248 - accuracy: 0.5377 - val_loss: 1.1212 - val_accuracy: 0.3776
Epoch 5/10
91/91 [==============================] - 4s 44ms/step - loss: 0.8353 - accuracy: 0.5838 - val_loss: 1.1646 - val_accuracy: 0.3505
Epoch 6/10
91/91 [==============================] - 4s 43ms/step - loss: 0.7604 - accuracy: 0.6438 - val_loss: 1.2138 - val_accuracy: 0.3588
Epoch 7/10
91/91 [==============================] - 4s 41ms/step - loss: 0.7025 - accuracy: 0.6753 - val_loss: 1.2575 - val_accuracy: 0.3608
Epoch 8/10
91

In [175]:
predicted = model.predict(np.asarray(pre_X(df_all['Tweet'])))

print(classification_report(np.argmax(one.fit_transform(np.asarray(polarity).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.35      0.96      0.51      4186
           1       0.00      0.00      0.00      4740
           2       0.57      0.16      0.25      3700

    accuracy                           0.37     12626
   macro avg       0.31      0.37      0.25     12626
weighted avg       0.28      0.37      0.24     12626



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## TASH

In [176]:
predicted = model.predict(np.asarray(pre_X(text_TASH)))

print(classification_report(np.argmax(one.fit_transform(np.asarray(sentiment_).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.34      0.88      0.49       881
           1       0.00      0.00      0.00      1018
           2       0.47      0.26      0.33       888

    accuracy                           0.36      2787
   macro avg       0.27      0.38      0.27      2787
weighted avg       0.26      0.36      0.26      2787



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## KANSOAN

In [177]:
predicted = model.predict(np.asarray(pre_X(text_KANSOAN)))

print(classification_report(np.argmax(Y_KANSOAN, axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.60      0.94      0.73      4504
           2       0.88      0.40      0.55      4720

    accuracy                           0.66      9224
   macro avg       0.74      0.67      0.64      9224
weighted avg       0.74      0.66      0.64      9224

